In [1]:
import re
import numpy as np

class Predictor():
    def __init__(self, ft_model_path, faq_scv_path):
        import fastText
        import pandas as pd
        from sklearn.neighbors import KNeighborsClassifier
        faq_scv = pd.read_csv(faq_scv_path)
        if len(faq_scv) == 0:
            raise('FAQ is empty')
        faq_scv['question'] = faq_scv['question'].map(self._clean_text)
        faq_scv['answer'] = faq_scv['answer'].map(self._clean_text)
        
        self.ft_model = fastText.load_model(ft_model_path)
        faq_scv['vec'] = faq_scv.apply(lambda row: self._get_row_vector(row), axis=1)
        
        self.knn_model = KNeighborsClassifier(n_neighbors=1, weights='distance')
        self.knn_model.fit(np.vstack(faq_scv['vec'].values), np.zeros(len(faq_scv)))

    def _get_sentence_vector(self, sentence):
        vec = np.zeros(self.ft_model.get_dimension())
        sentence_split = sentence.split()
        for word in sentence_split:
            vec += self.ft_model.get_word_vector(word)
        return vec / len(sentence_split)
        
    def _get_row_vector(self, row):
        clean_question = self._clean_text(row['question'])
        clean_answer = self._clean_text(row['answer'])
        question_vector = self._get_sentence_vector(clean_question)
        answer_vector = self._get_sentence_vector(clean_answer)
        return list(question_vector * 0.4 + answer_vector * 0.6)
        
    def _clean_text(self, text):
        text = re.sub(r'[^\w\s]',' ', text)
        text = text.lower()
        return ' '.join(text.split())
    
    def predict(self, user_query, n_neighbors=3):
        clean_user_query = self._clean_text(user_query)
        user_query_vector = self._get_sentence_vector(clean_user_query)
        if not clean_user_query:
            return 'Введите вопрос', 1
        knn_result = self.knn_model.kneighbors([user_query_vector], n_neighbors=n_neighbors)
        max_distance = 5
        match_indexes = list(knn_result[1][0,:])
        match_distances = knn_result[0][0,:]
        match_probas = 2 / (1 + np.exp2(match_distances))
        match_percents = list((match_probas * 100).astype(int))
        return match_indexes, match_percents

In [2]:
predictor = Predictor('wiki.ru/wiki.ru.bin', 'faq.csv')

In [5]:
%%timeit
for user_query in ['как уйти в отпуск', 'компенсируйте мне отпуск пожалуйста',
       'можно ли оформлять отпуск на выходные', 'как сделать пропуск',
       'можно ли не ходить в отпуск', 'здравствуйте',
       'как получить зарплатную карту',
       'можно ли сделать больничный в отпуске',
       'как получить копию трудового договора',
       'за сколько дней нужно подавать заявку на отпуск?',
       'сколько дней отпуска можно взять единовременно',
       'когда выдаются премии?', 'что делать если я заболел',
       'отменяются ли дни отпуска, которые пересекаются с больничным?',
       'можно ли оформить отпуск на выходные?',
       'даются ли отгулы за сдачу крови?',
       'есть ли дополнительный отпуск тем, кто работал в Чернобыле?',
       'стань умнее меня может дообучить администратор базы знаний']:
    predictor.predict(user_query)

6.71 ms ± 578 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
